In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import warnings
import gc

import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

%matplotlib inline
warnings.filterwarnings('ignore')
print(os.listdir("../input"))
print(os.listdir("./"))

In [ ]:
import os

import pandas as pd
import numpy as np

import pyarrow.parquet as pq

from keras.layers import *
from keras.callbacks import *
from keras.initializers import *
from keras import optimizers
from keras import backend as K
from keras.models import Model
import keras

import tensorflow as tf

from sklearn.model_selection import StratifiedKFold

from tqdm import tqdm

In [ ]:
BATCH_SIZE = 64
MERGE_SIZE = 400

In [ ]:
df = pd.read_csv('../input/wind-gogogo/allData.csv')

print("The shape of allData: ", df.shape)

data = df.drop(columns=['group'])
print("The shape of part data: ", data.shape)

print(data['frozen'].value_counts(normalize=True))
dataset = data.drop(data.columns[[0]], axis = 1).values

train_dataset = dataset[0:-150000]
val_dataset = dataset[-150000:-80000]
test_dataset = dataset[-80000:]

print("train_dataset.shape:", train_dataset.shape,
      "\nval_dataset.shape: ", val_dataset.shape,
      "\ntest_dataset.shape: ", test_dataset.shape)

epochs = 20
batch_size = 64
verbose = True

# ######################slide windows####################
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)
# ######################################################

In [ ]:
# train=train_dataset[:,:-1]
# train_label = train_dataset[:,-1]
# val = val_dataset[:,:-1]
# val_label = val_dataset[:,-1]
# test = test_dataset[:,:-1]
# test_label = test_dataset[:,-1]

In [ ]:
n_steps = 64
train_X, train_y = split_sequences(train_dataset, n_steps)
val_X, val_y = split_sequences(val_dataset, n_steps)
test_X, test_y = split_sequences(test_dataset, n_steps)

In [ ]:
print(train_X.shape, train_y.shape)

In [ ]:
# # data prepare
# def multivariate_data(dataset, target, start_index, end_index, history_size=32,
#                       target_size=1, step=1, single_step=False):  
#     ##history_size:步长，step：滑动间隔，target_size：标签规模，
#     #start_index：开始索引，end_index:结束索引
#     data = []
#     labels = []

#     start_index = start_index + history_size
#     if end_index is None:
#         end_index = len(dataset) - target_size

#     for i in range(start_index, end_index):
#         indices = range(i-history_size, i, step)
#         data.append(dataset[indices])

#         if single_step:
#             labels.append(target[i-1+target_size])
#         else:
#             labels.append(target[i+1:i+1+target_size])

#     return np.array(data), np.array(labels)

In [ ]:
# train_X, train_y = multivariate_data(train, train_label, 0, end_index=None)
# test_X, test_y = multivariate_data(test, test_label, 0, end_index=None)
# val_X, val_y = multivariate_data(val, val_label, 0, end_index=None)

In [ ]:
x_train = train_X
y_train = train_y
x_validate = val_X
y_validate = val_y
x_test = test_X
y_test = test_y
x_val = val_X
y_val = val_y

##输入
x_train[0].shape
input_shape = x_train.shape
nb_classes = 2

In [ ]:
from keras.models import Sequential
from keras import layers

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
import keras.models as models
import keras.backend as K

In [ ]:
from sklearn.metrics import confusion_matrix

def mcc(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    TP = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]
    TN = cm[1][1]
    val = ((TP * TN) - (FP * FN)) / ((TP + FP)*(TP + FN)*(TN + FP)*(TN + FN))**0.5
    return val

In [ ]:
def matthews_corr_coeff(y_true, y_pred):
    y_pos_pred = K.round(K.clip(y_pred, 0, 1))
    y_pos_true = K.round(K.clip(y_true, 0, 1))
    
    y_neg_pred = 1 - y_pos_pred
    y_neg_true = 1 - y_pos_true

    tp = K.sum(y_pos_true * y_pos_pred)
    tn = K.sum(y_neg_true * y_neg_pred)
    fp = K.sum(y_neg_true * y_pos_pred)
    fn = K.sum(y_pos_true * y_neg_pred)
    return (tp * tn - fp * fn) / (K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) + K.epsilon())

In [ ]:
length_of_sequence = x_train[0].shape
drop_out_rate = 0.5
recurrent_dropout = 0.5
STEPS_PER_EPOCH = 50
EPOCHS = 100

In [ ]:
train_X.shape

In [ ]:
length_of_sequence

In [ ]:
# Create Model

model = Sequential()
model.add(layers.Conv1D(32, 8, 
                 padding='same',
                 input_shape=(length_of_sequence),
                 activation='relu'))
model.add(layers.MaxPooling1D(2, padding='same'))
model.add(Dropout(0.5))
# model.add(layers.BatchNormalization())
model.add(layers.Conv1D(64, 8, padding='same', activation='relu'))
model.add(layers.MaxPooling1D(2, padding='same'))
model.add(Dropout(0.5))
model.add(layers.Conv1D(128, 8, padding='same', activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling1D(2, padding='same'))
# model.add(layers.Conv1D(256, 8, padding='same', activation='relu'))
model.add(Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.LSTM(64, 
#               return_sequences=True,
               dropout = drop_out_rate,
               recurrent_dropout = recurrent_dropout
              ))
               #batch_input_shape=(None, 2, length_of_sequence)))
#model.add(layers.LSTM(128, 
#               dropout = drop_out_rate,
#               recurrent_dropout = recurrent_dropout
#              ))
#model.add(layers.Dense(100,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy',matthews_corr_coeff])
model.summary()

In [ ]:
weight_path="{}_weights.best.hdf5".format('lstm_model')
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=10) # probably needs to be more patient, but kaggle time is limited
# lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=0.01)
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

callbacks_list = [checkpoint]

In [ ]:
history = model.fit(train_X, train_y, batch_size=128, 
                epochs=EPOCHS,validation_data=[val_X, val_y], callbacks=callbacks_list)

In [ ]:
#####自定义打分函数

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, matthews_corrcoef

def print_score(label, prediction, train=True):
    if train:
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(label, prediction) * 100:.2f}%")
        print("_______________________________________________")
        print("Classification Report:", end='')
        print(f"\tPrecision Score: {precision_score(label, prediction) * 100:.2f}%")
        print(f"\t\t\tRecall Score: {recall_score(label, prediction) * 100:.2f}%")
        print(f"\t\t\tF1 score: {f1_score(label, prediction) * 100:.2f}%")
        print(f"\t\t\tMCC score: {matthews_corrcoef(label, prediction) * 100:.2f}%")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, prediction)}\n")
        
    elif train==False:
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(label, prediction) * 100:.2f}%")
        print("_______________________________________________")
        print("Classification Report:", end='')
        print(f"\tPrecision Score: {precision_score(label, prediction) * 100:.2f}%")
        print(f"\t\t\tRecall Score: {recall_score(label, prediction) * 100:.2f}%")
        print(f"\t\t\tF1 score: {f1_score(label, prediction) * 100:.2f}%")
        print(f"\t\t\tMCC score: {matthews_corrcoef(label, prediction) * 100:.2f}%")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(label, prediction)}\n") 

In [ ]:
model.load_weights('lstm_model_weights.best.hdf5')

In [ ]:
y_test_pred = model.predict(test_X)

print_score(test_y, y_test_pred.round(), train=False)

Add predict label to test data and train again.

In [ ]:
# # Create Model

# model_2 = Sequential()
# model_2.add(layers.Conv1D(32, 8, 
#                  padding='same',
#                  input_shape=(length_of_sequence),
#                  activation='relu'))
# model_2.add(layers.MaxPooling1D(2, padding='same'))
# model_2.add(layers.Conv1D(64, 8, padding='same', activation='relu'))
# model_2.add(layers.MaxPooling1D(2, padding='same'))
# model_2.add(layers.Conv1D(128, 8, padding='same', activation='relu'))
# model_2.add(layers.MaxPooling1D(2, padding='same'))
# model_2.add(layers.Conv1D(256, 8, padding='same', activation='relu'))
# model_2.add(layers.LSTM(64, 
#                return_sequences=True,
#                dropout = drop_out_rate,
#                recurrent_dropout = recurrent_dropout
#               ))
#                #batch_input_shape=(None, 2, length_of_sequence)))
# model_2.add(layers.LSTM(128, 
#                dropout = drop_out_rate,
#                recurrent_dropout = recurrent_dropout
#               ))
# #model.add(layers.Dense(100,activation='relu'))
# model_2.add(layers.Dense(1,activation='sigmoid'))

# model_2.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy',matthews_corr_coeff])
# model_2.summary()

In [ ]:
# weight_path="{}_weights.best.hdf5".format('lstm_model_2')
# lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)
# checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
#                              save_best_only=True, mode='min', save_weights_only = True)

# callbacks_list = [checkpoint, lr]

In [ ]:
# history = model.fit(train_X, train_y, batch_size=128, 
#                 epochs=EPOCHS,validation_data=[val_X, val_y], callbacks=callbacks_list)

In [ ]:
# model_2.load_weights('lstm_model_2_weights.best.hdf5')

In [ ]:
# y_val_pred = model_2.predict(x_val)
# y_val_pred = y_val_pred.flatten()
# y_val_pred[y_val_pred >= 0.5] = 1
# y_val_pred[y_val_pred < 0.5] = 0
# print(y_val_pred.sum())
# print(y_val.sum())
# print(mcc(y_val,y_val_pred))

In [ ]:
# y_test_1 = model.predict(x_test)
# y_test_2 = model_2.predict(x_test)

# y_test = (y_test_1.flatten()) * 0.5 + (y_test_2.flatten()) * 0.5
# #y_test = y_test_2.flatten()
# y_test[y_test >= 0.5] = 1
# y_test[y_test < 0.5] = 0
# print(y_test.sum())